In [2]:
!pip install tensorflow

In [3]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [4]:
import tensorflow_datasets as tfds

(train_ds, val_ds), ds_info = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)


In [5]:
IMG_SIZE = 160

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))  # resize all images to 160x160
    image = image / 255.0  # normalize pixel values between 0 and 1
    return image, label


In [6]:
train_ds = train_ds.map(preprocess).batch(32).shuffle(1000)
val_ds = val_ds.map(preprocess).batch(32)


In [7]:
#frozen layers
base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                         include_top=False,
                         weights='imagenet')
base_model.trainable = False


In [8]:
#→ This is the pre-trained MobileNetV2 (with all layers frozen).
#→ It extracts meaningful features from images.
x = base_model.output
#→ This flattens the output from the CNN to prepare it for classification.
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
#This is the classifier layer.
#It takes the features and classifies the image (e.g., "daisy" vs "not daisy").
predictions = Dense(5, activation='softmax')(x)


In [9]:
model = Model(inputs=base_model.input, outputs=predictions)


In [10]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [11]:
model.fit(train_ds, validation_data=val_ds, epochs=5)


Epoch 1/5


92/92 ━━━━━━━━━━━━━━━━━━━━ 49s 461ms/step - accuracy: 0.6945 - loss: 0.8498 - val_accuracy: 0.8774 - val_loss: 0.3404
Epoch 2/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 38s 413ms/step - accuracy: 0.9269 - loss: 0.2356 - val_accuracy: 0.8856 - val_loss: 0.3327
Epoch 3/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 40s 426ms/step - accuracy: 0.9526 - loss: 0.1452 - val_accuracy: 0.8856 - val_loss: 0.3187
Epoch 4/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 41s 439ms/step - accuracy: 0.9797 - loss: 0.0810 - val_accuracy: 0.8896 - val_loss: 0.3457
Epoch 5/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 38s 410ms/step - accuracy: 0.9911 - loss: 0.0473 - val_accuracy: 0.8896 - val_loss: 0.3426
